# Image Preprocessing and Binary Classification with Keras

## Objective

In this week's exercise, you will:
1. Learn how to do image preprocessing in Keras
2. Build a multilayer neural network for binary classification
3. Train the model on a real-world dataset of cats and dogs
4. Monitor performance using a validation dataset

---

## Step 1: Import Libraries

Let's start by importing the necessary libraries.

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, Input, MaxPooling2D, Rescaling

---

## Step 2: Load and Preprocess the Data

Use `tfds.load()` to load the "cats_vs_dogs" dataset.

Find a way to split the dataset into a training and a validation set.

Also research how to apply necessary preprocessing to the data and do so (some of the preprocessing can also later be done using layers of the model).

*Note*: You can also get the dataset from other sources. However, there are some known issues with corrupted images, which then need to be addressed.

In [2]:
# TODO: load the dataset
def load_data():
    dataset, info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True)
    train_data, test_data = dataset['train'], dataset['test']
    return train_data, test_data, info

---

## Step 3: Build a Multilayer Neural Network

Build a multilayer neural network for binary classification. Apply your knowledge from the Coursera lectures to choose an adequate model architecture.

In [3]:
# TODO build a model
model = Sequential([
    Input(shape=(128, 128, 3)),
    Rescaling(1./255),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
# TODO compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


---

## Step 4: Train the Model

Train the model using the training dataset you created. Monitor performance during training using the validation dataset.

In [4]:
# TODO: train the model
def train_model(train_data, test_data):
    BATCH_SIZE = 32
    EPOCHS = 10

    train_data = train_data.map(lambda x, y: (tf.image.resize(x, (128, 128)), y))
    test_data = test_data.map(lambda x, y: (tf.image.resize(x, (128, 128)), y))

    train_data = train_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    test_data = test_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    history = model.fit(train_data, epochs=EPOCHS, validation_data=test_data)
    return history


---

## Step 5: Evaluate the Model

After training, you may upload some test images to evaluate your model.

In [8]:
import ipywidgets as widgets
from IPython.display import display
from tensorflow.keras.preprocessing import image
from PIL import Image
import io

def load_and_predict(model):
    uploader = widgets.FileUpload(accept='image/*', multiple=True)
    display(uploader)

    def on_upload_change(change):
        for filename, fileinfo in uploader.value.items():
            img = Image.open(io.BytesIO(fileinfo['content']))
            img = img.resize((128, 128))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0) / 255.0

            classes = model.predict(x)
            result = "a dog" if classes[0] > 0.5 else "a cat"
            print(f'The model predicts that the image is of {result}')

    uploader.observe(on_upload_change, names='value')

# Call the function to upload images and get predictions
load_and_predict(model)

FileUpload(value=(), accept='image/*', description='Upload', multiple=True)